# Introduction

![NYC Skyline](nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

# Project statement

As a consultant working for a real estate start-up, you have collected Airbnb listing data from various sources to investigate the short-term rental market in New York. You'll analyze this data to provide insights on private rooms to the real estate company.

There are three files in the `data` folder: `airbnb_price.csv`, `airbnb_room_type.xlsx`, and `airbnb_last_review.tsv`.

- What are the dates of the earliest and most recent reviews? Store these values as two separate variables with your preferred names.
- How many of the listings are private rooms? Save this into any variable.
- What is the average listing price? Round to the nearest two decimal places and save into a variable.
- Combine the new variables into one DataFrame called `review_dates` with four columns in the following order: `first_reviewed`, `last_reviewed`, `nb_private_rooms`, and `avg_price`. The DataFrame should only contain one row of values.

# Importing packages, reading data

In [1]:
import pandas as pd
import numpy as np
import re
import itertools as it

In [2]:
price = pd.read_csv('data/airbnb_price.csv')
room_type = pd.read_excel('data/airbnb_room_type.xlsx')
last_review = pd.read_csv('data/airbnb_last_review.tsv', sep = '\t')

# Cleaning

For this project I do not need the neighborhood info from `price`; the description from `room_type`; or the host name from `last_review`. Further I will only use the listing id columns to verify that the three DataFrames are referring to the same sample of listings.

## data: price

In [3]:
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   price        25209 non-null  object
 2   nbhood_full  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB


So `price` isn't missing any values. However it's weird that the `'price'` column is of type `object`. Let's look at that:

In [4]:
price['price'].head()

0    225 dollars
1     89 dollars
2    200 dollars
3     79 dollars
4    150 dollars
Name: price, dtype: object

So it looks like the prices are of the form '[int] dollars'. Let me confirm that:

In [5]:
# series of bools:
#   True if the price format matches
#   False if not
is_dollar_format = price['price'].apply(    
    lambda x : bool(
        re.match('[\d]+ dollars', x)
    )
)

# Find number of rows which DON'T match the price format
print('Number of rows with different price format:')
price[~is_dollar_format].shape[0]

Number of rows with different price format:


0

N.B. the above also confirms that all are the prices are ints, since I didn't include the possibility of a decimal point in the regex.

I can therefore deal with this by just stripping off the `' dollars'` and converting to `int`:

In [6]:
price['price'] = price['price'].str.replace(' dollars', '').astype('int')

In [7]:
price['price'].describe()

count    25209.000000
mean       141.777936
std        147.349137
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       7500.000000
Name: price, dtype: float64

The minimum (zero dollars?) and maximum (7500 dollars???) seem a little out there, but the 25-75% range (69-175) seems reasonable enough.

## data: room_type

In [8]:
room_type.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   description  25199 non-null  object
 2   room_type    25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB


The room type column has no missing values, but lots of value inconsistency:

In [9]:
room_type['room_type'].unique()

array(['Entire home/apt', 'private room', 'Private room',
       'entire home/apt', 'PRIVATE ROOM', 'shared room',
       'ENTIRE HOME/APT', 'Shared room', 'SHARED ROOM'], dtype=object)

But all the value inconsistency is in the case, so I'll just lowercase everything and call it a day.

In [10]:
room_type['room_type'] = room_type['room_type'].str.lower().astype('category')

In [11]:
room_type['room_type'].value_counts()

room_type
entire home/apt    13266
private room       11356
shared room          587
Name: count, dtype: int64

## data: last_review

In [12]:
last_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   host_name    25201 non-null  object
 2   last_review  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB


No missing values in the last review column:

In [13]:
last_review['last_review'].head()

0     May 21 2019
1    July 05 2019
2    June 22 2019
3    June 24 2019
4    June 09 2019
Name: last_review, dtype: object

And since the dates come with the months spelled out and the full year, I don't have to worry about format ambiguity. So:

In [14]:
last_review['last_review'] = pd.to_datetime(last_review['last_review'])

Taking a look at the output:

In [15]:
last_review['last_review'].dt.year.agg(['min', 'max'])

min    2019
max    2019
Name: last_review, dtype: int64

In [16]:
last_review['last_review'].dt.month.agg(['min', 'max'])

min    1
max    7
Name: last_review, dtype: int64

So all the reviews seem to be from between January and July of 2019.

## listing_id: consistency check

Now to check that the listing id's are consistent. I'll do this by evaluating whether each DataFrame's set of unique listing id's is contained in the others' sets:

In [17]:
price_id = price['listing_id'].unique()
room_type_id = room_type['listing_id'].unique()
last_review_id = last_review['listing_id'].unique()

id_arrays = [price_id, room_type_id, last_review_id]

for x in it.permutations(id_arrays, r=2):
    print(
        set(x[0]).issubset(set(x[1]))
    )

True
True
True
True
True
True


All true, so every listing id is represented in all of the DataFrames.

# Project solution

## Earliest and latest reviews

In [18]:
last_review['last_review'].agg(['min', 'max'])

min   2019-01-01
max   2019-07-09
Name: last_review, dtype: datetime64[ns]

In [19]:
first_reviewed = last_review['last_review'].min()
last_reviewed = last_review['last_review'].max()

## Number of private rooms

In [20]:
nb_private_rooms = room_type['room_type'].value_counts()['private room']

## Average listing price

In [21]:
avg_price = round(price['price'].mean(), 2)

## Submission DataFrame

Combine the new variables into one DataFrame called `review_dates` with four columns in the following order: `first_reviewed`, `last_reviewed`, `nb_private_rooms`, and `avg_price`. The DataFrame should only contain one row of values.

In [22]:
review_dates = pd.DataFrame()
review_dates['first_reviewed'] = [first_reviewed]
review_dates['last_reviewed'] = [last_reviewed]
review_dates['nb_private_rooms'] = [nb_private_rooms]
review_dates['avg_price'] = [avg_price]

In [23]:
review_dates

,first_reviewed,last_reviewed,nb_private_rooms,avg_price
0,2019-01-01,2019-07-09,11356,141.78
